In [1]:
#Install beautifulsoop this is used to pass the html code and find details inside it.
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=f5a3e643ea4bf3339efe93dee9ae4b812e3b0af3d602f1ea9ed4d35e5461d3e1
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [3]:
#Install  request  this is used to send request from our pc to amazon website
!pip install requests

In [4]:
!pip install pandas

In [66]:
 from bs4 import BeautifulSoup
 import pandas as pd
 import requests
 import numpy as np

In [6]:
# we need to send request from local pc to amazon website and get the html code. For this we need URL (for the particular product we are extracting html code, here we are extracting tv data)
URL = "https://www.amazon.com/s?k=tv+qled+65&crid=3G7585ZN8705J&sprefix=tv+qled+65%2Caps%2C167&ref=nb_sb_noss_1"


In [7]:
# http header(this is for user agent). we got this from 'www.whatismybrowser.com' select 'detect my settings' select 'what is my user agent' we will get the link.
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0', 'Accept-Language':'en-US, en;q=0.5'})

In [8]:
#http request
webpage = requests.get(URL,headers=HEADERS)

In [11]:
#Response 200 is succesful
webpage

<Response [200]>

In [26]:
type(webpage.content)

bytes

In [13]:
#Soup object containing all the content , this will convert content into html page.
soup=BeautifulSoup(webpage.content,"html.parser")

In [30]:
#find different links on the page , mostly links are inside the anchor tag. Fetch links as list of tag objects.
links = soup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [33]:
#to get links from links variable.
link = links[0].get('href')

In [35]:
productlist = "https://amazon.com" + link
productlist

'https://amazon.com/TCL-65Q650G-Accelerator-Streaming-Television/dp/B0C1J5228L/ref=sr_1_1?crid=3G7585ZN8705J&keywords=tv+qled+65&qid=1697223414&sprefix=tv+qled+65%2Caps%2C167&sr=8-1'

In [51]:
# using above link as new url and sending request
webpage1 = requests.get(productlist,headers=HEADERS)

In [52]:
webpage1

<Response [200]>

In [53]:
newsoup=BeautifulSoup(webpage1.content,"html.parser")

In [60]:
#to get product title
newsoup.find("span",attrs={"id":"productTitle"}).text.strip()

'TCL 65-Inch Q6 QLED 4K Smart TV with Google (65Q650G, 2023 Model) Dolby Vision, Atmos, HDR Pro+, Game Accelerator Enhanced Gaming, Voice Remote, Works Alexa, Streaming UHD Television'

In [62]:
#to get price
newsoup.find("span",attrs={"class":"a-price-whole"}).text

'498.'

In [63]:
# to get rating
newsoup.find("span",attrs={"class":"a-icon-alt"}).text

'4.2 out of 5 stars'

In [64]:
# Function to extract Product Title price and rating for entire URL
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [67]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=tv+qled+65&crid=3G7585ZN8705J&sprefix=tv+qled+65%2Caps%2C167&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)